In [1]:
!pip install beautifulsoup4
!pip install lxml
!pip install requests

In [1]:
from bs4 import BeautifulSoup
import requests
import logging
import http.client
http.client.HTTPConnection.debuglevel = 1
import pandas as pd
import time
import re
import os

In [2]:
# List of Hotel's URL
list_hotel = pd.read_excel('fix_hotel.xlsx', sheet_name='fix_hotel')
list_hotel.info()
list_hotel = list_hotel.loc[1194:1243, :]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1289 entries, 0 to 1288
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   hotel         1289 non-null   object
 1   hotel_link    1289 non-null   object
 2   review_count  1289 non-null   int64 
 3   hotel_name    1289 non-null   object
dtypes: int64(1), object(3)
memory usage: 40.4+ KB


In [3]:
list_hotel

,hotel,hotel_link,review_count,hotel_name
1194,FuramaXclusive Ocean Beach,https://www.tripadvisor.co.id/Hotel_Review-g60...,1313,furamaxclusive_
1195,TS Suites Seminyak,https://www.tripadvisor.co.id/Hotel_Review-g46...,1321,_ts_suites_semi
1196,FuramaXclusive Resort & Villas Ubud,https://www.tripadvisor.co.id/Hotel_Review-g40...,1325,furamaxclusive_
1197,Harper Kuta,https://www.tripadvisor.co.id/Hotel_Review-g29...,1333,_harper_kuta
1198,Lv8 Resort Hotel,https://www.tripadvisor.co.id/Hotel_Review-g31...,1339,lv8_resort_hote
1199,The Lovina,https://www.tripadvisor.co.id/Hotel_Review-g15...,1343,_the_lovina
1200,Peppers Seminyak,https://www.tripadvisor.co.id/Hotel_Review-g13...,1358,_peppers_seminy
1201,Hotel Horison Seminyak,https://www.tripadvisor.co.id/Hotel_Review-g46...,1379,_hotel_horison_
1202,Kamandalu Ubud,https://www.tripadvisor.co.id/Hotel_Review-g29...,1397,kamandalu_ubud
1203,Aksari Resort Ubud by Ini Vie Hospitality,https://www.tripadvisor.co.id/Hotel_Review-g41...,1397,aksari_resort_u


In [4]:
logging.basicConfig()
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)
requests_log.propagate = True

In [5]:
def hotelScraper():
    for hotel in list_hotel.index:
        hotel_name = list_hotel['hotel'][hotel]
        hotel_url = list_hotel['hotel_link'][hotel]
        filename = list_hotel['hotel_name'][hotel]
        filepath = 'Dataset_1/' + filename + '.csv'

        if os.path.exists(filepath):
            continue
        
        
        print(f"Start Scrapping: {hotel_name}")
        
        gathered_review = []
        headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
        pages = 200
        pages = (pages * 5) + 5
        for i in range(5, pages, 5):
            new_url = re.sub('or5', 'or'+str(i), str(hotel_url))
            url = requests.get(new_url, headers=headers)

            soup = BeautifulSoup(url.text, 'lxml')
            reviews = soup.find_all('div', class_= 'YibKl MC R2 Gi z Z BB pBbQr')
            
            if reviews:
                for review in reviews:
                    user_text = review.find('span', class_= 'QewHA H4 _a').text
                    user_rating = review.find('span', class_= 'ui_bubble_rating')['class'][1][7]
                    gathered_review.append([hotel_name, user_text, user_rating])

                #Pages Processed
                cpages = int(i/5)
                print(f"Done Scrapping {hotel_name} page {cpages}")
                print('\n')
                time.sleep(3)

                # Make csv file
                df = pd.DataFrame(gathered_review, columns=['hotel', 'review', 'stars'])
                df.to_csv('Dataset_1/'+filename+'.csv')
            else:
                if os.path.exists(filepath):
                    break
                else:
                    hotelScraper()
    
        

In [7]:
hotelScraper()